## Step 1.1: Install Packages and Libraries

In [4]:
!pip install mesh_to_sdf
# !apt-get install xvfb
!pip install pyvirtualdisplay

In [5]:
"""
Step 1.1: Install Necessary Packages and Libraries
"""

from IPython import get_ipython
from IPython.display import display

import torch
from torch import nn
from mesh_to_sdf import sample_sdf_near_surface
import trimesh
from torch.utils.data import DataLoader, Dataset
import numpy as np
from math import sqrt
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

## Step 1.2 Prepare Training Dataset 

In [ ]:
"""
Step 1.2: Prepare the Training Dataset from Input Mesh
"""

class NeuralSDFDataset(Dataset):
    def __init__(self, mesh_path, sample_num, device='cuda'):
        """
        In this function, we first use a package called `trimesh` (it's already imported in Step 1.1) to load an `.obj` file with path <code>mesh_path</code>
        We then sample sample_num points around the surface by calling method `sample_sdf_near_surface`.
        
        Your task is to convert the sampled points and their sdf values (with the type of `numpy ndarray`) to torch tensors by calling the `torch.from_numpy` function.
        After conversion, you will send those tensors to CUDA GPU by calling the `.to(device)` function.
        The converted device tensors should be stored in self.points and self.sdf in separate.
        """
        mesh = trimesh.load(mesh_path)
        points, sdf = sample_sdf_near_surface(mesh, number_of_points=sample_num)

        ### you implementation starts
        
        ### you implementation ends


    def __len__(self):
        return 1 # we are not using this

    def __getitem__(self, idx):
        return self.points, self.sdf

In [ ]:
"""
This block is a checkpoint for you Step 1.2 implementation. Run the block to check the plot of the sample point distribution and make sure it is consistent with the input shape.
There is no implementation requirement within this block. 
"""

### Helper method for test result of the sampled points from your dataset class.
def test_dataset(sdf_loader_test):
  points, sdf = next(iter(sdf_loader_test))
  points =  points.cpu().detach().numpy().squeeze(0)
  sdf = sdf.cpu().detach().numpy().squeeze()
  norm = plt.Normalize(vmin=np.min(sdf), vmax=np.max(sdf))
  colors = plt.cm.coolwarm(norm(sdf))
  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111, projection='3d')

  sc = ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=sdf, cmap='coolwarm', marker='o')

  cbar = plt.colorbar(sc, ax=ax, shrink=0.5)
  cbar.set_label("SDF Value")

  ax.set_xlabel("X")
  ax.set_ylabel("Y")
  ax.set_zlabel("Z")
  ax.set_title("3D Point Cloud Visualization with SDF Values")
  ax.view_init(elev=0, azim=0)
  plt.show()

  
sample_num = 10000
device='cuda'
mesh_path="cow.obj" ### Change to bunny.obj if needed.

sdf_test = NeuralSDFDataset(mesh_path, sample_num, device=device)
sdf_loader_test = DataLoader(sdf_test, num_workers=0)
test_dataset(sdf_loader_test)

## Step 1.3 Network Structure

In [ ]:
"""
Step 1.3: Neural Network Structure for SDF Representation
"""

class SineLayer(nn.Module):
    """
    Default sin activation frequency w0 is set to be 30, feel free to play with it.
    However, we set this to be 15 by default due to our network is much smaller that suffers from learning high frequency features.
    If you have time, make the hidden layers to 512 width with 5 depth, then checkout the difference.

    By default, the weights for the first layer are initialized differently as suggested in Sec.3.2 in the original paper. We use is_first flag to
    check whether we should init the weights differently.

    We use linear layer as the last layer without any activation functions since SDF values shouldn't be limited to a certain range.
    We use is_last flag to check if we should use activation functions or not.
    """

    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, is_last=False, w0=15, skip_weight=1):
        """
        In this function, you are tasked to initialize the fully-connectd layer self.fc using the feature vectors with their sizes specified by in_featuers and out_features
        """
        
        super().__init__()
        self.w0 = w0                         # a float specifying the default frequency in activation function 
        self.is_first = is_first             # a boolean flag indicating if the layer is the first layer
        self.is_last = is_last               # a boolean flag indicating if the layer is the last layer
        self.skip_weight = skip_weight       # a float weight controlling skip connection
        self.in_features = in_features       # an integer specifying the size of the input feature vector
        self.out_features = out_features     # an integer specifying the size of the output feature vector
        self.fc = None                       # fully connected layer; None as default

        ### your implementation starts
                
        ### your implementation ends

        self.init_weights()

    def init_weights(self):
        """
        This function initializes the weights for the first layer and other layers (see details in the Deep SDF paper Sec.3.2).
        No implementation is required in this function.
        """
        with torch.no_grad():
            if self.is_first:
                self.fc.weight.uniform_(-1. / self.in_features,
                                             1. / self.in_features)
            else:
                self.fc.weight.uniform_(-np.sqrt(6 / self.in_features) / self.w0,
                                             np.sqrt(6 / self.in_features) / self.w0)

    def forward(self, x):
        """
        You are tasked to implement the activation function by using the output of the fully connected layer taking x.
        The implementation should consists of three cases: the first layer, the last layer, and the intermediate layer(s).
            - If the layer is the first layer, you should apply the sine activation function to the output of the fully connected layer with w0 as its frequency;
            - If the layer is the last layer, you should take the output of the fully connected layer as the final output;
            - If the layer is an intermediate layer, you should add the output from the sine activation function weighted by skip_weight to the original x.
        """
        ### your implementation starts
        
        ### your implementation ends

class NeuralSDF(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, w0=30):
        super().__init__()

        """
        You are tasked to initialize all the layers in the neural network, including the first layer, the intermediate layer(s), and the last layer.
        The initialized network layers will be stored in the list of nn.
        Make sure to use the input arguments of the init function when initializing these layers. 
        """

        self.network = []                               # a list storing all the layers; empty by default
        self.w0 = w0                                    # a float specifying the activation function frequency 
        self.hidden_features = hidden_features          # an integer specifying the size of the hidden-layer feature vector
        self.hidden_layers = hidden_layers              # an integer specifying specifying the number of hidden layers
        self.in_features = in_features                  # an integer specifying the size of the input feature vector
        self.out_features = out_features                # an integer specifying the size of the output feature vector

        ### your implementation starts
        
        ### your implementation ends
        
        self.network = nn.Sequential(*self.network)

    def forward(self, x):
        output = self.network(x)
        return output

## Step 1.4 Train Your Network

In [ ]:
"""
Step 1.4: Train Your Neural Network with Adam Optimizer
"""
def train_neuralSDF(dataloader, hidden_features, hidden_layers, w0, lr=1e-4, iterations=10000, device='cuda'):
    """
    You are tasked to implement the training loop of the neural network. 
    For each epoch, you will start with a zero gradient and use the Mean Squared Loss (MSE) as your loss function. 
    Then, you need to propagate the loss backward and run the optimization step function provided by the optimizer.
    """

    model = NeuralSDF(in_features=3, out_features=1, hidden_features=hidden_features, hidden_layers=hidden_layers, w0=w0).to(device)
    optimizer = torch.optim.Adam(lr=lr, params=model.parameters(), weight_decay=.0)
    data, labels = next(iter(dataloader))

    for epoch in range(iterations):

        ### your implementation starts
        
        ### your implementation ends
        
        if epoch % 500 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

    return model

In [ ]:
"""
sample_num: total points sampled (feel free to increase this if needed)
mesh_path: relative path to .obj file location
"""

sample_num = 300000  ### total number of points sampled as training points, feel free to change this.
device='cuda'
mesh_path="cow.obj" ### mesh path to your mesh,

sdf = NeuralSDFDataset(mesh_path, sample_num, device=device)
sdfloader = DataLoader(sdf, num_workers=0)

In [ ]:
"""
hidden_features: hidden layer width.
hidden_layers: hidden layer depth.
w0: Activation frequency. We suggest 15 for our given examples.

Feel free to play around with these parameters.
"""
hidden_features = 16 ### hidden layer width, feel free to change
hidden_layers = 2 ### hidden layer depth, feel free to change
w0 = 15 ### activation function frequency, feel free to change
iterations = 10000 ### total number of training iterations, feel free to change
lr = 1e-4 ### learning rate, feel free to change

neural_sdf = train_neuralSDF(sdfloader, hidden_features = hidden_features, hidden_layers = hidden_layers, w0 = w0, lr=lr, iterations=iterations, device=device)

## Step 2 Copy Network Weights to Shader 

In [ ]:

"""
Run this step to generate the text file for the neural network weights. 
The generated weights will be printed to the Notebook output.
There is no implementation requirement for this section.

The neural SDF to ShaderToy conversion were modified based on Blackle Mori's Neural Stanford Bunny: https://www.shadertoy.com/view/wtVyWK
"""

import re

### Helper function for convert pytorch cuda tensor to numpy arrays
def dump_data(dat):
  dat = dat.cpu().detach().numpy()
  return dat

### Print a vector to a form that's usable in fragement shader
def print_vec4(ws):
  vec = "vec4(" + ",".join(["{0:.2f}".format(w) for w in ws]) + ")"
  vec = re.sub(r"\b0\.", ".", vec)
  return vec

### Print a matrix to a form that's usable in fragement shader
def print_mat4(ws):
  mat = "mat4(" + ",".join(["{0:.2f}".format(w) for w in np.transpose(ws).flatten()]) + ")"
  mat = re.sub(r"\b0\.", ".", mat)
  return mat

### Since we know networks are just matrices and vectors, this function converts our network to matrices and vectors that 
### can be compiled in fragement shader. 
def serialize_to_shadertoy(network, varname):
  omega = network.w0
  chunks = int(network.hidden_features/4)
  lin = network.network[0].fc
  in_w = dump_data(lin.weight)
  in_bias = dump_data(lin.bias)
  om = omega
  for row in range(chunks):
    line = "vec4 %s0_%d=sin(" % (varname, row)
    for ft in range(network.in_features):
        feature = x_vec = in_w[row*4:(row+1)*4,ft]*om
        line += ("p.%s*" % ["y","z","x"][ft]) + print_vec4(feature) + "+"
    bias = in_bias[row*4:(row+1)*4]*om
    line += print_vec4(bias) + ");"
    print(line)

  #hidden layers
  for layer in range(network.hidden_layers):
    layer_w = dump_data(network.network[layer+1].fc.weight)
    layer_bias = dump_data(network.network[layer+1].fc.bias)
    for row in range(chunks):
      line = ("vec4 %s%d_%d" % (varname, layer+1, row)) + "=sin("
      for col in range(chunks):
        mat = layer_w[row*4:(row+1)*4,col*4:(col+1)*4]*omega
        line += print_mat4(mat) + ("*%s%d_%d"%(varname, layer, col)) + "+\n    "
      bias = layer_bias[row*4:(row+1)*4]*omega
      line += print_vec4(bias)+")/%0.1f+%s%d_%d;"%(sqrt(layer+1), varname, layer, row)
      print(line)

  #output layer
  out_w = dump_data(network.network[-1].fc.weight)
  out_bias = dump_data(network.network[-1].fc.bias)
  for outf in range(network.out_features):
    line = "return "
    for row in range(chunks):
      vec = out_w[outf,row*4:(row+1)*4]
      line += ("dot(%s%d_%d,"%(varname, network.hidden_layers, row)) + print_vec4(vec) + ")+\n    "
    print(line + "{:0.3f}".format(out_bias[outf])+";")

In [ ]:
serialize_to_shadertoy(neural_sdf, 'f')